In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from datetime import datetime
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import pickle

In [3]:
initial_data = pd.read_csv('./data/frenchLanguage.txt', delimiter = "\t")
print(initial_data.shape)
initial_data.head()

(436493, 47)


,Row,Sample Name,Transaction Id,Anon Student Id,Session Id,Time,Time Zone,Duration (sec),Student Response Type,Student Response Subtype,...,KC Category (Default).2,KC (Default).3,KC Category (Default).3,KC (Single-KC),KC Category (Single-KC),KC (Unique-step),KC Category (Unique-step),School,Class,CF (media message)
0,1,All Data,6578b254e514b10664996b383d18f628,Stu_0297622ac301b5800c639eb2c0d12337,6637162180020c6600b77406c5441e3f,2007-10-03 14:14:32,UTC,0,START_TUTOR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University of Pittsburgh,NaN,NaN
1,2,All Data,28ba2bd6ce11a6c15d67c4664094f135,Stu_0297622ac301b5800c639eb2c0d12337,6637162180020c6600b77406c5441e3f,2007-10-03 14:14:32,UTC,0,VIDEO_ACTION,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University of Pittsburgh,NaN,NaN
2,3,All Data,3f5d211cee5917340b1ef1db9794be0c,Stu_0297622ac301b5800c639eb2c0d12337,6637162180020c6600b77406c5441e3f,2007-10-03 14:14:51,UTC,19,VIDEO_ACTION,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University of Pittsburgh,NaN,NaN
3,4,All Data,638794be3a4bdd41d06f2965900bc5d5,Stu_0297622ac301b5800c639eb2c0d12337,6637162180020c6600b77406c5441e3f,2007-10-03 14:15:34,UTC,0,VIDEO_ACTION,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University of Pittsburgh,NaN,NaN
4,5,All Data,a1c7ab4643029119929f1f2f2c5ef198,Stu_0297622ac301b5800c639eb2c0d12337,6637162180020c6600b77406c5441e3f,2007-10-03 14:15:34,UTC,0,START_TUTOR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University of Pittsburgh,NaN,NaN


In [12]:
initial_data = initial_data.dropna(subset = ['Outcome'])
print(initial_data.shape)

(256255, 47)


In [13]:
len(initial_data['Problem Name'].unique())

1245

In [15]:
len(initial_data['Anon Student Id'].unique())

99

In [14]:
initial_data['Anon Student Id'].value_counts()

Stu_e46aa25adef44f4fc674a0b388066b5f    14746
Stu_8d68ffbd410e069b1351786d13e50c5f    11838
Stu_dbfbf7461c726ddbc7be6ace5048ea42     8192
Stu_7d95599eebe224dd0e8cb4a0df675891     7670
Stu_b40ba2afbd73a26158c33410475825c3     7416
                                        ...  
Stu_fd9c0f0c3b1ec429b15e5c515beb1147       11
Stu_f0ce95411ec7eed8b5ac6e16a7181674        8
Stu_fc04f6fcafe9592d0fb8bebe71f1ad2d        3
Stu_6732f23fa2b8812a7b0475d90b7163d5        2
Stu_73cd12dcefde132df73fe02d1f937ada        1
Name: Anon Student Id, Length: 99, dtype: int64

In [3]:
initial_data.columns

Index(['Row', 'Sample Name', 'Transaction Id', 'Anon Student Id', 'Session Id',
       'Time', 'Time Zone', 'Duration (sec)', 'Student Response Type',
       'Student Response Subtype', 'Tutor Response Type',
       'Tutor Response Subtype', 'Level (Unit)', 'Level (Section)',
       'Problem Name', 'Problem View', 'Problem Start Time', 'Step Name',
       'Attempt At Step', 'Is Last Attempt', 'Outcome', 'Selection',
       'Selection.1', 'Selection.2', 'Action', 'Input', 'Input.1', 'Input.2',
       'Feedback Text', 'Feedback Classification', 'Help Level',
       'Total Num Hints', 'KC (Default)', 'KC Category (Default)',
       'KC (Default).1', 'KC Category (Default).1', 'KC (Default).2',
       'KC Category (Default).2', 'KC (Default).3', 'KC Category (Default).3',
       'KC (Single-KC)', 'KC Category (Single-KC)', 'KC (Unique-step)',
       'KC Category (Unique-step)', 'School', 'Class', 'CF (media message)'],
      dtype='object')

In [8]:
needed = initial_data[['Anon Student Id','Problem Name','Student Response Type','Step Name','Attempt At Step', 'Is Last Attempt','Outcome']]
needed

,Anon Student Id,Problem Name,Student Response Type,Step Name,Attempt At Step,Is Last Attempt,Outcome
0,Stu_0361c37e8eb8a731484f34e1ab9dfdcd,LDEMO_WKST,CYCLE,R1C2,1,1,OK
1,Stu_0361c37e8eb8a731484f34e1ab9dfdcd,LDEMO_WKST,CYCLE,R1C1,1,1,OK
2,Stu_0361c37e8eb8a731484f34e1ab9dfdcd,LDEMO_WKST,CYCLE,R2C2,1,1,OK
3,Stu_0361c37e8eb8a731484f34e1ab9dfdcd,LDEMO_WKST,CYCLE,R2C1,1,1,OK
4,Stu_0361c37e8eb8a731484f34e1ab9dfdcd,LDEMO_WKST,CYCLE,R4C1,1,0,BUG
...,...,...,...,...,...,...,...
436809,Stu_fee81c4391766153444ccaf109a5a20c,TRANSFORMN021,CYCLE,CurrFunctionText,1,0,OK
436810,Stu_fee81c4391766153444ccaf109a5a20c,TRANSFORMN021,CYCLE,GRAPHICAL-TRANSFORMATIONS-refl-v,1,1,OK
436811,Stu_fee81c4391766153444ccaf109a5a20c,TRANSFORMN021,CYCLE,ALGEBRAIC-TRANSFORMATIONS-refl-v,1,1,OK
436812,Stu_fee81c4391766153444ccaf109a5a20c,TRANSFORMN021,CYCLE,TransformButton-refl-v,1,1,OK


In [9]:
len(needed['Anon Student Id'].unique())

59

# Collecter les données

In [3]:
initial_data.columns

Index(['Row', 'Anon Student Id', 'Problem Hierarchy', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)'],
      dtype='object')

In [4]:
initial_data = initial_data.rename(columns={'Anon Student Id': 'user_id','Problem Name': 'item_id','Step Name': 'step_id','KC(Default)': 'kc_id','First Transaction Time': 'timestamp','Correct First Attempt': 'correct',"Hints":'nb_hint','Incorrects':'incorrect','Corrects':'incorrect_?'})
needed = initial_data[["user_id","item_id","timestamp","correct","nb_hint"]]
needed

,user_id,item_id,timestamp,correct,nb_hint
0,0BrbPbwCMz,EG4-FIXED,2005-09-09 12:24:49.0,0,3
1,0BrbPbwCMz,EG4-FIXED,2005-09-09 12:25:31.0,1,0
2,0BrbPbwCMz,EG40,2005-09-09 12:25:43.0,0,3
3,0BrbPbwCMz,EG40,2005-09-09 12:26:34.0,1,0
4,0BrbPbwCMz,EG40,2005-09-09 12:28:36.0,1,0
...,...,...,...,...,...
809689,IQQo3367x0,EG40,2006-03-09 10:58:13.0,1,0
809690,IQQo3367x0,EG40,2006-03-09 10:58:18.0,1,0
809691,IQQo3367x0,EG40,2006-03-09 10:58:22.0,0,3
809692,IQQo3367x0,EG40,2006-03-09 10:58:44.0,1,0


In [5]:
needed['timestamp'] = pd.to_datetime(needed['timestamp']).map(lambda t: t.timestamp()).round().astype(np.int32)

# Conservez uniquement les utilisateurs avec au moins 10 interactions
needed = needed.groupby("user_id").filter(lambda x: len(x) >= 10)

# Supprimer les entrées où le résultat n'est pas binaire mais continu
needed = needed[needed.correct.isin([0, 1])]
needed['correct'] = needed['correct'].astype(np.int32)

# Trier la colonne "timestamp" et changer le nom de la colonne
needed = needed.sort_values('timestamp')
print(needed.shape)
needed.head()

(808821, 5)


,user_id,item_id,timestamp,correct,nb_hint
535958,45TTYcotWk,LDEMO_WKST,1125395408,0,0
562642,HvghFVBS5v,LDEMO_WKST,1125395410,1,0
569598,IQ98biy227,LDEMO_WKST,1125395414,1,0
555691,Dmq6441349,LDEMO_WKST,1125395416,1,0
545705,80nlN05JQ6,LDEMO_WKST,1125395419,0,0


# Encodage

In [77]:
userId_encoder = LabelEncoder()
itemId_encoder = LabelEncoder()

needed["user_id"] = userId_encoder.fit_transform(needed["user_id"])
needed["item_id"] = itemId_encoder.fit_transform(needed["item_id"])

# Save user_id and item_id encoder
####
outputUserInEncoder = open('userId_encoder.pkl', 'wb')
pickle.dump(userId_encoder, outputUserInEncoder)
outputUserInEncoder.close()
####
outputItemInEncoder = open('itemId_encoder.pkl', 'wb')
pickle.dump(itemId_encoder, outputItemInEncoder)
outputItemInEncoder.close()

needed.head()

### Load saved encoder
"""
import pickle 
pkl_file = open('Departure_encoder.pkl', 'rb')
le_departure = pickle.load(pkl_file) 
pkl_file.close()
df_test['Departure'] = le_departure.transform(df_test['Departure'])
"""

"\nimport pickle \npkl_file = open('Departure_encoder.pkl', 'rb')\nle_departure = pickle.load(pkl_file) \npkl_file.close()\ndf_test['Departure'] = le_departure.transform(df_test['Departure'])\n"

In [78]:
# replace user_id and item_id where id == 0
needed["user_id"].replace(0,len(needed['user_id'].unique()),inplace=True)
needed["item_id"].replace(0,len(needed['item_id'].unique()),inplace=True)

In [79]:
print(f"items = {len(needed['item_id'].unique())} and users = {len(needed['user_id'].unique())}")

items = 1084 and users = 569


In [80]:
conditions = [
    (needed['correct'] == 1) & (needed['nb_hint'] > 0),
    (needed['correct'] == 1) & (needed['nb_hint'] == 0),
    (needed['correct'] == 0) & (needed['nb_hint'] > 0),
    (needed['correct'] == 0) & (needed['nb_hint'] == 0)
    ]
# 1 ==> Correct avec hint
# 2 ==> Correct sans hint
# 3 ==> Incorrect avec hint
# 4 ==> Incorrect sans hint
values = [1, 2, 3, 4]

# create a new column and use np.select to assign values to it using our lists as arguments
needed['answers_using_hint'] = np.select(conditions, values)
needed.head()

,user_id,item_id,timestamp,correct,nb_hint,answers_using_hint
535958,72,563,1125395408,0,0,4
562642,247,563,1125395410,1,0,2
569598,249,563,1125395414,1,0,2
555691,212,563,1125395416,1,0,2
545705,154,563,1125395419,0,0,4


In [76]:
## dataConfig 
users = len(needed['user_id'].unique())
items = len(needed['item_id'].unique())
n_observations = len(needed)
config = {
    'user_ids': users,
    'item_ids': items,
    'observations': n_observations,
}
config

{'user_ids': 569, 'item_ids': 1084, 'observations': 809660}

In [81]:
#Save configuration to csv file
needed.to_csv("./data/neededData.csv")